In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from keras.preprocessing.image import load_img,img_to_array

C:\Users\mridu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
infected='./cell_images/Parasitized/'
uninfected='./cell_images/Uninfected/'

In [3]:
train_infected=os.listdir(infected)
train_uninfected=os.listdir(uninfected)

In [4]:
print(len(train_infected))
print(len(train_uninfected))

13780
13780


In [5]:
X_val1=train_infected[13700:13779]
print(len(X_val1))
X_val2=train_uninfected[13700:13779]
print(len(X_val2))
X_val=[]
for i in X_val1:
    X_val.append(i)
for i in X_val2:
    X_val.append(i)
print(len(X_val))
y_val=[]
for i in range(158):
    if(i<79):
        y_val.append("Infected")
    else:
        y_val.append("Not Infected")
print(len(y_val))


79
79
158
158


In [6]:
X_train1=train_infected[:12700]
X_train2=train_uninfected[:12700]
X_train=[]
for i in X_train1:
    X_train.append(i)
for i in X_train2:
    X_train.append(i)
print(len(X_train))
y_train=[]
for i in range(25400):
    if(i<12700):
        y_train.append("Infected")
    else:
        y_train.append("Not Infected")
print(len(y_train))

25400
25400


In [7]:
X_test1=train_infected[12700:13700]
X_test2=train_uninfected[12700:13700]
X_test=[]
for i in X_test1:
    X_test.append(i)
for i in X_test2:
    X_test.append(i)
print(len(X_test))
y_test=[]
for i in range(2000):
    if(i<1000):
        y_test.append("Infected")
    else:
        y_test.append("Not Infected")
print(len(y_test))

2000
2000


In [8]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [9]:
infected+X_train[0]

'./cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png'

In [10]:
height=100
width=100
channels=3
actualX_train=None
actualX_train=np.ndarray(shape=(len(X_train),height,width,channels),dtype=np.float32)
print("Shape of Training Data: ",actualX_train.shape)
var=0
for i in X_train:
    if(var<12700):
        img=load_img(infected+i)
    else:
        img=load_img(uninfected+i)
    img=img.resize((width,height),Image.ANTIALIAS)
    x=img_to_array(img)
    actualX_train[var]=x
    var+=1

Shape of Training Data:  (25400, 100, 100, 3)


In [11]:
height=100
width=100
channels=3
actualX_test=np.ndarray(shape=(len(X_test),height,width,channels),dtype=np.float32)
print("Shape of Testing Data: ",actualX_test.shape)
var=0
for i in X_test:
    if(var<1000):
        img=load_img(infected+i)
    else:
        img=load_img(uninfected+i)
    img=img.resize((width,height),Image.ANTIALIAS)
    x=img_to_array(img)
    actualX_test[var]=x
    var+=1

Shape of Testing Data:  (2000, 100, 100, 3)


In [12]:
height=100
width=100
channels=3
actualX_val=np.ndarray(shape=(len(X_val),height,width,channels),dtype=np.float32)
print("Shape of Validation Data: ",actualX_val.shape)
var=0
for i in X_val:
    if(var<79):
        img=load_img(infected+i)
    else:
        img=load_img(uninfected+i)
    img=img.resize((width,height),Image.ANTIALIAS)
    x=img_to_array(img)
    actualX_val[var]=x
    var+=1

Shape of Validation Data:  (158, 100, 100, 3)


In [13]:
target_series=pd.Series(y_train)
one_hot=pd.get_dummies(target_series,sparse=True)
y_train=np.asarray(one_hot)

In [14]:
target_series=pd.Series(y_test)
one_hot=pd.get_dummies(target_series,sparse=True)
y_test=np.asarray(one_hot)

In [15]:
target_series=pd.Series(y_val)
one_hot=pd.get_dummies(target_series,sparse=True)
y_val=np.asarray(one_hot)

In [21]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.layers import GlobalAveragePooling2D


model=Sequential()
model.add(Conv2D(filters=32,kernel_size=5,padding='same',activation='relu',input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=32,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=5,activation='relu'))
model.add(MaxPooling2D(pool_size=2))


model.add(Conv2D(filters=32,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 100, 32)      2432      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 64)        51264     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 32)        51232     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 32)        0         
__________

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
type(actualX_val[0])

numpy.ndarray

In [24]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpointer=ModelCheckpoint(filepath='model.malaria.hdf5',verbose=1,save_best_only=True)
early_stop=EarlyStopping(monitor='val_loss',patience=4,verbose=1)
cnn=model.fit(actualX_train,y_train,batch_size=100,epochs=10,validation_data=(actualX_val,y_val),callbacks=[checkpointer,early_stop],verbose=1,shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 25400 samples, validate on 158 samples
Epoch 1/10
25400/25400 [==============================] - 11711s 461ms/step - loss: 0.4589 - acc: 0.7719 - val_loss: 0.3270 - val_acc: 0.9304

Epoch 00001: val_loss improved from inf to 0.32696, saving model to model.malaria.hdf5
Epoch 2/10
25400/25400 [==============================] - 1656s 65ms/step - loss: 0.1592 - acc: 0.9487 - val_loss: 0.2547 - val_acc: 0.9304

Epoch 00002: val_loss improved from 0.32696 to 0.25470, saving model to model.malaria.hdf5
Epoch 3/10
25400/25400 [==============================] - 1673s 66ms/step - loss: 0.1439 - acc: 0.9514 - val_loss: 0.1680 - val_acc: 0.9494

Epoch 00003: val_loss improved from 0.25470 to 0.16803, saving model to model.malaria.hdf5
Epoch 4/10
25400/25400 [==============================] - 1497s 59ms/step - loss: 0.1368 - acc: 0.9550 - val_loss: 0.2088 - val_acc: 0.9430

Epoch 00004: val_loss did not improve from 0.16803
Epoch 5/10
25400/2

In [25]:
model.load_weights('model.malaria.hdf5')

In [26]:
score=model.evaluate(actualX_test,y_test,verbose=0)
score[1]

0.949

In [60]:
import time

In [ ]:
print("Enter The Path Of The X Ray IMage of your cell")
path=input()
print("[INFO] Give Me A Second ..  Im working on it")
a=np.ndarray(shape=(1,100,100,3),dtype=np.float32)
img=load_img(path)
img=img.resize((100,100),Image.ANTIALIAS)
x=img_to_array(img)
x=x.reshape((1,100,100,3))
a=x
prediction=model.predict(a)
prediction=prediction.reshape(-1).tolist()
time.sleep(5.0)
if(prediction[0]>prediction[1]):
    print("Malaria Cells Detected,Consult Your Doctor")
else:
    print("No Worries! You are Not infected!")

Enter The Path Of The X Ray IMage of your cell
